In [5]:
from datetime import timedelta
from datetime import datetime
import csv
import time
import re
import requests
#import numpy as np
from urllib.parse import urlencode
#import selenium
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import   By 
from selenium.webdriver.firefox.options  import   Options 
from selenium.webdriver.support import   expected_conditions  as   EC 
from selenium.webdriver.support.wait  import   WebDriverWait 
from selenium.common.exceptions       import NoSuchElementException as NSE #gestione delle eccezioni
from selenium.common.exceptions       import TimeoutException as TOE
from selenium.common.exceptions       import StaleElementReferenceException as SERE
from selenium.common.exceptions       import ElementClickInterceptedException as ECIE

In [6]:
# COSTANTI
italia=[{'regione':'Abruzzo',               'provincie': ['L\'Aquila','Teramo','Pescara','Chieti']},
        {'regione':'Basilicata',            'provincie': ['Potenza','Matera']},
        {'regione':'Calabria',              'provincie': ['Cosenza','Catanzaro','Crotone','Vibo Valentia','Reggio Calabria']},
        {'regione':'Campania',              'provincie': ['Caserta','Benevento','Avellino','Salerno','Napoli']},
        {'regione':'Emilia-Romagna',        'provincie': ['Piacenza','Parma','Reggio Emilia','Modena','Ferrara','Ravenna','Bologna','Rimini','Forlì']},
        {'regione':'Friuli Venezia Giulia', 'provincie': ['Udine','Gorizia','Trieste','Pordenone']},
        {'regione':'Lazio',                 'provincie': ['Viterbo','Rieti','Latina','Frosinone','Roma']},
        {'regione':'Liguria',               'provincie': ['Imperia','Savona','La Spezia','Genova']},
        {'regione':'Lombardia',             'provincie': ['Bergamo','Brescia','Como','Cremona','Lecco','Lodi','Mantova','Milano','Varese','Monza','Sondrio','Pavia']},
        {'regione':'Marche',                'provincie': ['Ancona','Macerata','Ascoli Piceno','Fermo','Pesaro']},
        {'regione':'Molise',                'provincie': ['Campobasso','Isernia']},
        {'regione':'Piemonte',              'provincie': ['Asti','Alessandria','Biella','Vercelli','Novara','Cuneo','Torino','Verbania']},
        {'regione':'Puglia',                'provincie': ['Foggia','Taranto','Brindisi','Lecce','Bari','Andria']},
        {'regione':'Sardegna',              'provincie': ['Sassari','Nuoro','Oristano','Cagliari','Carbonia']},
        {'regione':'Sicilia',               'provincie': ['Trapani','Agrigento','Caltanissetta','Enna','Ragusa','Siracusa','Palermo','Messina','Catania']},
        {'regione':'Toscana',               'provincie': ['Lucca','Pistoia','Livorno','Pisa','Arezzo','Siena','Grosseto','Prato','Firenze','Massa']},
        {'regione':'Trentino-Alto Adige',   'provincie': ['Bolzano','Trento']},
        {'regione':'Umbria',                'provincie': ['Perugia','Terni']},
        {'regione':'Valle d\'Aosta',        'provincie': ['Aosta']},
        {'regione':'Veneto',                'provincie': ['Verona','Vicenza','Belluno','Treviso','Padova','Rovigo','Venezia']}  
]
attesaCorta=4 #variabili attesa esplicita caricamento pagina 
attesa=8
ignoredExceptions=(NSE, SERE, TOE, ECIE)

In [7]:
# FUNZIONI

def preparaDriver():
    options = Options()
    options.headless = True  #modalita senza interfaccia grafica
    options.add_argument('--lang=it') #imposto la lingua del browser in italiano
    options.set_preference("permissions.default.image", 2) #disablita lo scaricamento delle immagini
    driver = Firefox(options=options) #invoco il webdriver di firefox geckodriver in path
    return driver


def scrape_risultati(driver, delta, dataOdierna, regione, citta, numstruttureTrovate):
    
    try:
        #clickCookie=driver.find_element_by_xpath('//*[@id="cookie_warning"]/div/div/div[2]/button/span').click()
        clickCookie=WebDriverWait(driver, attesa).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cookie_warning"]/div/div/div[2]/button/span')))
        driver.execute_script("arguments[0].click();", clickCookie)

    except:
        print('NoCookie')
        
    try:
        time.sleep(attesaCorta)
        click=WebDriverWait(driver, attesa).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="filter_label"][contains(text(),"Hotel")]')))
        time.sleep(attesa)
        driver.execute_script("arguments[0].click();", click)
    
    except ignoredExceptions:
        print("errori NSE TOE SERE ECIE")
        #clickHotel = WebDriverWait(driver, attesa).until(EC.element_to_be_clickable((By.XPATH, '//span[@class="filter_label"][contains(text(),"Hotel")]')))
        #time.sleep(attesa)
        #driver.execute_script("arguments[0].click();", clickHotel)
        #driver.find_element_by_xpath('//span[@class="filter_label"][contains(text(),"Hotel")]')
        pass
    
    time.sleep(attesaCorta) #caricamento pagina necessario
    alloggiCitta = list() #lista dove salvare i risultati

    for accomodation in driver.find_elements_by_class_name('sr_item_content'):
        Hotel=dict() #dizionario per appendere le informazioni dei singoli hotel
        # nomeHotel = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div[1]/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]').text #restituisce il nome hotel
        try:
            nomeHotel = accomodation.find_element_by_class_name('sr-hotel__name').text
            #print("Hotel: " + nomeHotel)
            htmlText  = accomodation.get_attribute('innerHTML')
    
        except ignoredExceptions:
            print("errore nel caricamento del DOM nella pagina hotel")
            WebDriverWait(driver, attesa).until(EC.presence_of_element_located((By.CLASS_NAME, 'sr-hotel__name')))
            nomeHotel = accomodation.find_element_by_class_name('sr-hotel__name').text
            htmlText  = accomodation.get_attribute('innerHTML')
        
        prezzoStrutture=(re.findall('(€&nbsp;\d*)+', htmlText)[0]).replace("€&nbsp;","")
        Hotel['regione'           ]=regione         
        Hotel['citta'             ]=citta  
        Hotel['data acquisizione' ]=dataOdierna
        Hotel['distanza giorni'   ]=delta
        Hotel['disponibilita'     ]=numstruttureTrovate
        Hotel['nome Hotel'        ]=nomeHotel
        Hotel['prezzo'            ]=prezzoStrutture
        alloggiCitta.append(Hotel)
        
    return alloggiCitta   
    

#if __name__ == '__main__':
def main():    
    # datetime object containing current date and time
    now         = datetime.now()
    dataOdierna = now.strftime("%d/%m/%Y")
    nomeFile    = now.strftime("%Y%m%d-%H%M")+'_booking.csv'
    alloggi = []
    driver = preparaDriver()

    # predispongo il file CSV dove salvare i dati inserendo la riga iniziale
    keys = ['regione','citta','data acquisizione','distanza giorni','disponibilita','nome Hotel','prezzo']
    
    with open(nomeFile, 'a', newline='') as csvfile:
        dict_writer = csv.DictWriter(csvfile, keys)
        dict_writer.writeheader()

        # Ciclo su tutti gli intervalli di tempo che sono 4 
       
        for delta in [1, 7, 14, 21, 28]: #funzione che permette di scandire per intervalli temporali
            print("\nDelta: " + str(delta))
            dataCheckin =now + timedelta(days=delta) #da  verificare se prende il settimo giorno
            dataCheckout=dataCheckin + timedelta(days=1)
            parametri = {
                        'checkin_month':    dataCheckin.month,
                        'checkin_monthday': dataCheckin.day,
                        'checkin_year':     dataCheckin.year,
                        'checkout_month':   dataCheckout.month,
                        'checkout_monthday':dataCheckout.day,
                        'checkout_year':    dataCheckout.year,
                        'group_adults':2,
                        'group_children':0,
                        'dest_type': 'city',
                        'ss': '',  
                        'order' : 'distance_from_search'
            }

            # ciclo su ogni città
            for regione in italia:
                nomeRegione = regione['regione']
                for city in regione['provincie']:
                    print(nomeRegione+"-"+city)        
                    
                    # Preparo l'url per interrogare booking
                    parametri['ss'] = city+", "+nomeRegione+", Italia"
                    queryString     = urlencode(parametri) 
                    url             ="https://www.booking.com/searchresults.html?%s" % queryString
                    driver.get(url)
                    wait = WebDriverWait(driver, attesa).until( #attesa implicita
                        EC.presence_of_all_elements_located((By.CLASS_NAME, 'sr-hotel__title')))

                    # recupero il numero di strutture disponibili
                    xpathStruttura = driver.find_element_by_xpath('//*[@role="heading"]').text  #percorso del titolo che varia a seconda delle citta
                    numstruttureTrovate=(re.findall('[\d]*[\.]?[\d]+',xpathStruttura)[0]) #prende il primo numero in lista relativo alle strutture

                    # eseguo lo scraping e salvo su file
                    #city=city.split(",")[0]
                    dict_writer.writerows(scrape_risultati(driver, delta, dataOdierna, nomeRegione, city, numstruttureTrovate))

            # Chiusura del driver
            driver.close()
            driver.quit()

In [8]:
main()


Delta: 1
Abruzzo-L'Aquila
Abruzzo-Teramo
Abruzzo-Pescara
Abruzzo-Chieti
Basilicata-Potenza
Basilicata-Matera
Calabria-Cosenza
Calabria-Catanzaro


KeyboardInterrupt: 